# OTP Generation and Verification System

This project aims to implement an OTP (One-Time Password) generation, verification, and validation system that integrates with a MySQL database and performs the following tasks:
- Generates a random 6-digit OTP.
- Simulates sending the OTP to a user's email address.
- Allows the user to input the OTP and verifies if the entered OTP is correct.
- Handles retry attempts in case of incorrect OTP entries.
- Logs OTP details (email, IP address, and country) to a MySQL database.

## Libraries Required
We will use the following libraries:
- `random`: For generating random OTP.
- `re`: To validate email format.
- `mysql.connector`: For database interaction.
- `requests`: To retrieve IP address and country information.

Let's begin by importing the necessary libraries.


In [40]:
# Import required libraries
import random
import re
import mysql.connector
from mysql.connector import Error
import requests


## Connecting to MySQL Database

The table schema should be as follows:
```sql
CREATE TABLE otp_table (
    email VARCHAR(255) PRIMARY KEY,
    otp VARCHAR(6),
    ip_address VARCHAR(50),
    country VARCHAR(50),
    otp_generated_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);


In [41]:
# Function to connect to MySQL
def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host="localhost",        
            user="root",             
            password="Harsha@159",  
            database="otp_db"        # Database name
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None


## OTP Generation

We will now implement a function `generate_otp()` that generates a 6-digit OTP randomly.


In [42]:
# Function to generate a 6-digit OTP
def generate_otp():
    otp = ''.join([str(random.randint(0, 9)) for _ in range(6)])
    return otp


## Email Validation

Next, we need to validate the email address input by the user.


In [43]:
# Function to validate email format using regex
def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(pattern, email) is not None


## Retrieving User's IP Address and Country

To store more information about the OTP request, we will retrieve the user's IP address and country based on their IP address.

We'll use the `requests` library to fetch this information.

### Steps:
1. **Get IP Address**: Use the `httpbin.org` service to get the public IP.
2. **Get Country**: Use the `ipinfo.io` service to get the country based on the IP.


In [44]:
# Function to get the user's IP address
def get_ip_address():
    response = requests.get("https://httpbin.org/ip")
    ip_info = response.json()
    return ip_info['origin']

# Function to get the country from IP address using ipinfo.io API
def get_country_from_ip(ip):
    url = f"http://ipinfo.io/{ip}/json"
    response = requests.get(url)
    data = response.json()
    return data.get("country", "Unknown")


## Storing OTP in the MySQL Database

Now we need a function `store_otp_in_db()` that will store the generated OTP along with the email address, IP address, and country in the MySQL database.

This function will insert a new record or update the existing record if the email already exists in the database.



In [45]:
# Function to store OTP in the MySQL database
def store_otp_in_db(email, otp):
    ip_address = get_ip_address()
    country = get_country_from_ip(ip_address)
    
    connection = None
    try:
        connection = connect_to_mysql()
        if connection:
            cursor = connection.cursor()
            # Insert or update OTP, IP address, and country for the email
            query = """
                INSERT INTO otp_table (email, otp, ip_address, country)
                VALUES (%s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE otp = %s, otp_generated_time = CURRENT_TIMESTAMP, ip_address = %s, country = %s
            """
            cursor.execute(query, (email, otp, ip_address, country, otp, ip_address, country))
            connection.commit()
            print(f"OTP for {email} stored in database with IP: {ip_address} and Country: {country}.")
    except Error as e:
        print(f"Error: {e}")
    finally:
        if connection:
            connection.close()


## Verifying OTP from the Database

The function `verify_otp_from_db()` will verify if the OTP entered by the user matches the OTP stored in the database for that email address.

If the OTP is correct, the user is granted access. If the OTP is incorrect, the user can retry.

In [46]:
# Function to verify OTP from the MySQL database
def verify_otp_from_db(email, entered_otp):
    connection = None
    try:
        connection = connect_to_mysql()
        if connection:
            cursor = connection.cursor()
            query = "SELECT otp FROM otp_table WHERE email = %s"
            cursor.execute(query, (email,))
            stored_otp = cursor.fetchone()

            if stored_otp and stored_otp[0] == entered_otp:
                print("OTP verified successfully. Access granted!")
                new_otp = generate_otp()
                store_otp_in_db(email, new_otp)  # Store the new OTP
                return True
            else:
                print("Incorrect OTP. Try again.")
                return False
    except Error as e:
        print(f"Error: {e}")
        return False
    finally:
        if connection:
            connection.close()


use otp_db  

    SELECT * FROM otp_table;

truncate otp_table; 

drop table  otp_table;  


## Main Function

Now, we can implement the main function `main()` which will tie everything together. This function will:
- Prompt the user for their email.
- Generate and store the OTP.
- Allow the user to enter the OTP and verify it.

The user will have a maximum of 3 attempts to enter the correct OTP.


# OTP Generation and Verification (Tkinter GUI)

In [47]:
import tkinter as tk
class OTPApp:
    def __init__(self, root):
        self.root = root
        self.root.title("OTP Verification System")

        # Font size configuration
        font_config = ('Arial', 14)  # Set base font size for widgets

        # Email input
        self.email_label = tk.Label(root, text="Enter your email:", font=font_config)
        self.email_label.pack(pady=5)
        self.email_entry = tk.Entry(root, width=30, font=font_config)
        self.email_entry.pack(pady=5)

        # Generate OTP button
        self.generate_button = tk.Button(root, text="Generate OTP", command=self.generate_otp, font=font_config)
        self.generate_button.pack(pady=10)

        # OTP input
        self.otp_label = tk.Label(root, text="Enter OTP:", font=font_config)
        self.otp_label.pack(pady=5)
        self.otp_entry = tk.Entry(root, width=30, font=font_config)
        self.otp_entry.pack(pady=5)

        # Submit button
        self.submit_button = tk.Button(root, text="Submit OTP", command=self.submit_otp, font=font_config)
        self.submit_button.pack(pady=10)

        # Message area
        self.message_label = tk.Label(root, text="", fg="red", font=('Arial', 16))
        self.message_label.pack(pady=10)

        # Store OTP for verification
        self.generated_otp = None
        self.attempts_left = 3  # Tracks number of attempts left

    def generate_otp(self):
        email = self.email_entry.get()

        if not is_valid_email(email):
            self.show_message("Invalid email address. Please enter a valid one.", "red")
            return

        # Generate OTP
        self.generated_otp = generate_otp()

        # Store OTP in the "database"
        store_otp_in_db(email, self.generated_otp)

        # Show OTP in a custom pop-up message box
        self.show_custom_message(f"Your OTP is: {self.generated_otp}")

        # Update message label
        self.show_message(f"OTP generated and stored for {email}. Please check your email.", "green")

        # Reset the number of attempts
        self.attempts_left = 3

    def submit_otp(self):
        email = self.email_entry.get()
        entered_otp = self.otp_entry.get()

        if not is_valid_email(email):
            self.show_message("Invalid email address. Please enter a valid one.", "red")
            return

        if self.attempts_left > 0:
            # Verify OTP from the database
            if verify_otp_from_db(email, entered_otp):
                self.show_message("OTP verified successfully. Access granted.", "green")
                self.attempts_left = 3  # Reset attempts on successful verification
            else:
                self.attempts_left -= 1
                if self.attempts_left == 0:
                    self.show_message("Maximum attempts reached. Access denied.", "red")
                    self.submit_button.config(state=tk.DISABLED)  # Disable the submit button after max attempts
                else:
                    self.show_message(f"Incorrect OTP. {self.attempts_left} attempt(s) left.", "red")
        else:
            self.show_message("Access Denied. Please generate a new OTP.", "red")

    def show_message(self, message, color):
        self.message_label.config(text=message, fg=color)

    def show_custom_message(self, message):
        # Create a custom pop-up window (Toplevel window)
        popup = tk.Toplevel(self.root)
        popup.title("Generated OTP")

        # Set a larger font size for the text
        font_config = ('Arial', 18)
        
        # Add a label with the OTP message
        otp_label = tk.Label(popup, text=message, font=font_config, padx=20, pady=20)
        otp_label.pack()

        # Add a button to close the pop-up window
        close_button = tk.Button(popup, text="Close", command=popup.destroy, font=('Arial', 14), padx=10, pady=5)
        close_button.pack(pady=10)

        # Set the dimensions of the pop-up window
        popup.geometry("400x200")  # You can adjust the size as needed

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = OTPApp(root)
    root.mainloop()


In [48]:
"""# Main function to handle OTP generation, storage, and verification
def main():
    email_address = input("Enter your email address: ")

    if not is_valid_email(email_address):
        print("Invalid email address. Please enter a valid one, like example@gmail.com")
        return

    generated_otp = generate_otp()

    store_otp_in_db(email_address, generated_otp)

    print(f"Sending OTP to {email_address}...")
    print(f"Your OTP is: {generated_otp}")
    print("Please use this OTP to complete your verification process.")

    max_attempts = 3
    attempts = 0

    while attempts < max_attempts:
        entered_otp = input("Enter the OTP sent to your email: ")
        if verify_otp_from_db(email_address, entered_otp):
            print("OTP verified successfully. Access granted.")
            break
        else:
            attempts += 1
            print(f"Incorrect OTP. You have {max_attempts - attempts} attempts left.")

        if attempts == max_attempts:
            print("Maximum attempts reached. Access denied.")

# Run the main function
if __name__ == "__main__":
    main()"""


'# Main function to handle OTP generation, storage, and verification\ndef main():\n    email_address = input("Enter your email address: ")\n\n    if not is_valid_email(email_address):\n        print("Invalid email address. Please enter a valid one, like example@gmail.com")\n        return\n\n    generated_otp = generate_otp()\n\n    store_otp_in_db(email_address, generated_otp)\n\n    print(f"Sending OTP to {email_address}...")\n    print(f"Your OTP is: {generated_otp}")\n    print("Please use this OTP to complete your verification process.")\n\n    max_attempts = 3\n    attempts = 0\n\n    while attempts < max_attempts:\n        entered_otp = input("Enter the OTP sent to your email: ")\n        if verify_otp_from_db(email_address, entered_otp):\n            print("OTP verified successfully. Access granted.")\n            break\n        else:\n            attempts += 1\n            print(f"Incorrect OTP. You have {max_attempts - attempts} attempts left.")\n\n        if attempts == max_